# Preambulo

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
# higher resolution figures
plt.rcParams['figure.dpi'] = 500
plt.rcParams['savefig.dpi'] = 500

TRAFFIC_PATH = 'traffic.mp4'

A continuación hay varias funciones para trabajar mejor con VideoCaptures.

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_video(video_path):
	video = cv2.VideoCapture(video_path)
	try:
		yield video
	finally:
		video.release()

def video_frames(video):
	while video.isOpened():
		ret, frame = video.read()
		if ret:
			yield frame
		else:
			break

def frames(video_path):
	with open_video(video_path) as video:
		yield from video_frames(video)

def frames_interractive(video_path, fps=None):
	with open_video(video_path) as video:
		if fps is None:
			fps = video.get(cv2.CAP_PROP_FPS)
		for frame in video_frames(video):
			yield frame
			k = cv2.waitKey(int(1000 / fps)) & 0xff
			if k == 27:
				break
		cv2.destroyAllWindows()

Primero se empieza con obtener una "media" de la imagen. El objetivo de esta es obtener la imagen de fondo de la carretera.
De forma de que cualquier coche que aparezca se contraste con esta imagen.

Sin saber el número de elementos en una serie $S$, se puede conseguir la media $X$ de esta con la siguiente fórmula:

\begin{align*}
  X_1 &= S_1 \\
  X_{n+1} &= X_{n} \cdot \frac{n}{n+1} + S_{n+1} \cdot \frac{1}{n+1}
\end{align*}

In [ ]:

video = cv2.VideoCapture(TRAFFIC_PATH)

_, first_frame = video.read()

first_frame = cv2.resize(first_frame, (0, 0), fx=0.8, fy=0.8)
first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)

n = 1
mean = first_frame.copy()

for frame in video_frames(video):
	frame = cv2.resize(frame, (0, 0), fx=0.8, fy=0.8)
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	n += 1
	mean = mean * (n - 1) / n + frame / n

mean = mean.astype(np.uint8)
plt.imshow(mean)
mean_gray = cv2.cvtColor(mean, cv2.COLOR_RGB2GRAY)


A continuación, para empezar solo trabajaremos con solo el primer frame del vídeo.

Convertimos este a escala de grises y mostramos la diferencia entre esta y la media.

In [ ]:
first_gray = cv2.cvtColor(first_frame, cv2.COLOR_RGB2GRAY)

first_gray = first_gray.astype(np.uint8)

diff = cv2.absdiff(first_gray, mean_gray)

plt.imshow(diff, cmap="gray")

A continuación, se aplica un umbral a la imagen para obtener una imagen binaria.
Además se dilata la imagen para que los coches se vean mejor, cerrando los huecos que se puedan haber creado.

In [ ]:
th = np.max(diff) / 4
_, tresh = cv2.threshold(diff, th, 255, cv2.THRESH_BINARY)
#kernel = np.ones((3, 3), np.uint8)
#dilated = cv2.dilate(tresh, kernel, iterations=1)
fig, ax = plt.subplots()

ax.imshow(tresh, cmap="gray")

Por último, se obtienen los contornos de la imagen binaria y se dibujan sobre la imagen original.

In [ ]:

# find the contours
contours, _ = cv2.findContours(tresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
img = first_frame.copy()

for cnt in contours:
	# get the bounding rect
	#cv2.drawContours(img, [cnt], 0, 255, -1)
	x, y, w, h = cv2.boundingRect(cnt)
	# draw a green rectangle to visualize the bounding rect
	img = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1)

plt.imshow(img, cmap='gray')

In [ ]:
def get_contour_bounds(frame, mean):
	diff = cv2.absdiff(frame, mean)
	th = np.max(diff) / 4
	_, tresh = cv2.threshold(diff, th, 255, cv2.THRESH_BINARY)
	# if we want to dilate the image, this would be the place to do it
	#kernel = np.ones((3, 3), np.uint8)
	#dilated = cv2.dilate(tresh, kernel, iterations=1)
	# get the bounding rects
	contours, _ = cv2.findContours(tresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
	for cnt in contours:
		yield np.array(cv2.boundingRect(cnt))


In [ ]:
from dataclasses import dataclass

def inc_g():
	i = 0
	while True:
		yield i
		i += 1

inc = inc_g()

@dataclass
class Car:
	pos: np.ndarray # shape: (F, 2) where F is the number of frames, the second dimension is the x and y coordinates
	template: np.ndarray # RGB image template

th = None

for frame in frames_interractive(TRAFFIC_PATH):
	frame = cv2.resize(frame, (0, 0), fx=0.8, fy=0.8)
	frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	diff = cv2.absdiff(frame_gray, mean_gray)
	th = np.max(diff) / 4 if th is None else th
	_, tresh = cv2.threshold(diff, th, 255, cv2.THRESH_BINARY)
	
	img = frame.copy()

	cnts,_ = cv2.findContours(tresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	for cnt in cnts:
		x, y, w, h = cv2.boundingRect(cnt)
		img = cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 1)
	
	cv2.imshow("trafic", img)
	cv2.imshow("diff", tresh)


# Final traffic tracking

* suffijo `b` para la mitad inferior de imagenes.
* suffijo `g` para imagenes en escala de grises.

In [ ]:

from dataclasses import dataclass

DISTANCE_THRESHOLD = 100
SIZE_TRESHOLD = 100

@dataclass
class Car:
	first_apparition: int
	pos: np.ndarray # shape: (F, 4) where F is the number of frames, the second dimension is the x y coordinates, and the width & height of the center of the bounding box
	template: np.ndarray # RGB image of the car

	def is_this_car(self, frame, pos):
		dist = np.linalg.norm(self.pos[-1, 0:2] - np.array(pos[0:2]))
		size = np.linalg.norm(self.pos[-1, 2:4] - np.array(pos[2:4]))

		return dist < DISTANCE_THRESHOLD and size < SIZE_TRESHOLD
	
	def update_car(self, frame, pos):
		self.pos = np.vstack((self.pos, pos))
		self.template = frame[pos[1]:pos[1]+pos[3], pos[0]:pos[0]+pos[2]]
	
	def find_car(self, frame):
		a = cv2.matchTemplate(frame, self.template, cv2.TM_CCOEFF_NORMED)




mean_b = mean[mean.shape[0]//2:, :, :]
mean_bg = cv2.cvtColor(mean_b, cv2.COLOR_RGB2GRAY)

th = None

fi = 0

car_list = []

def from_contours(cnts, fi):
	...


for frame in frames_interractive(TRAFFIC_PATH):
	frame = cv2.resize(frame, (0, 0), fx=0.8, fy=0.8)
	frame_b = frame[frame.shape[0]//2:, :, :]
	frame_bg = cv2.cvtColor(frame_b, cv2.COLOR_BGR2GRAY)
	diff = cv2.absdiff(frame_b, mean_b)
	diff = np.sum(diff, axis=2) // 3
	diff = diff.astype(np.uint8)
	th = np.max(diff) / 3 if th is None else th
	_, tresh = cv2.threshold(diff, th, 255, cv2.THRESH_BINARY)

	kernel = np.ones((3, 3), np.uint8)
	tresh = cv2.erode(tresh, kernel, iterations=2)
	tresh = cv2.dilate(tresh, kernel, iterations=4)

	img = frame_b.copy()

	car_updated = [False for _ in car_list]

	cnts,_ = cv2.findContours(tresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	for cnt in cnts:
		x, y, w, h = cv2.boundingRect(cnt)
		if w > 20 and h > 20:
			# check if this contour is already a car
			for i, car in (car for car, u in zip(enumerate(car_list), car_updated) if not u):
				if car.is_this_car(frame_b, (x, y, w, h)):
					car.update_car(frame_b, (x, y, w, h))
					car_updated[i] = True
					break
			else:
				car_list.append(Car(fi, np.array([[x, y, w, h]]), frame_b[y:y+h, x:x+w]))

	for car in (car for car, u in zip(car_list, car_updated) if not u and fi - car.first_apparition > 10):
		car.find_car(frame_b)

	img = cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 1)

	for car in car_list:
		x, y, w, h = car.pos[-1]
		img = cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 1)
	
	cv2.imshow("trafic", img)
	cv2.imshow("diff", tresh)
	fi += 1